In [63]:
import csv
import collections
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt 
import copy
import time
import pandas as pd
import json
import scipy.io
import mpu

In [31]:
type_to_emission = collections.defaultdict(float)
type_to_emission['Conventional Hydroelectric'] = 0
type_to_emission['Hydroelectric Pumped Storage'] = 0
type_to_emission['Petroleum Liquids'] = 1808.5
type_to_emission['Natural Gas Internal Combustion Engine'] = 1255
type_to_emission['Natural Gas Fired Combined Cycle'] = 1255
type_to_emission['Natural Gas Steam Turbine'] = 1255
type_to_emission['Natural Gas Fired Combustion Turbine'] = 1255
type_to_emission['Nuclear'] = 0
type_to_emission['Geothermal'] = 166.7
type_to_emission['Onshore Wind Turbine'] = 0
type_to_emission['Other Waste Biomass'] = 76
type_to_emission['Wood/Wood Waste Biomass'] = 76
type_to_emission['Landfill Gas'] = 1255
type_to_emission['Solar Photovoltaic'] = 145
type_to_emission['Solar Thermal without Energy Storage'] = 145
type_to_emission['Conventional Steam Coal'] = 1149
type_to_emission['Other Gases'] = 1255
type_to_emission['Batteries'] = 0
type_to_emission['Petroleum Coke'] = 1808.5
type_to_emission['Municipal Solid Waste'] = 884
type_to_emission['Other Natural Gas'] = 1255
type_to_emission['IMPORT'] = 884
type_to_emission['Synchronous Condenser'] = 884

In [167]:
num_buses = 8870
num_gen = 2149
num_lines = 10823
bus = scipy.io.loadmat('MATPOWER/bus.mat')['bus']
load = np.array(bus[:, 2], dtype=float).reshape((num_buses, 1))
#load = load[load != 0.0].reshape((num_buses, 1))
df = pd.read_csv("GIS/CATS_gens.csv")
df = df[df['Pmax'] != 0.0].to_numpy()
branch_ = scipy.io.loadmat('MATPOWER/branch.mat')['brach']
branch_from_bus = list(map(int, branch_[:, 0]-1))
branch_to_bus = list(map(int, branch_[:, 1]-1))
line_to_nodes = np.array(branch_[:, 0:2]-1, dtype=int)

In [154]:
f = open("pf_solution.json")
sol = json.load(f)
generators = [0]*2149
cost_vec = [0]*2149
power_generation = [0]*2149
carbon_emission_rate_vec = [0]*2149
branch_power_to = [0]*10823
branch_power_from = [0]*10823
for line, val in sol['solution']['gen'].items():
    if val['pg'] != 0.0:
        generators[int(line)-1] = df[int(line)-1][2]-1
        carbon_emission_rate_vec[int(line)-1] = type_to_emission[df[int(line)-1][3]]
        cost_vec[int(line)-1] = val['pg_cost']*100
        power_generation[int(line)-1] = val['pg']*100

for line, val in sol['solution']['branch'].items():
    branch_power_from[int(line)-1] = val['pf']*100
    branch_power_to[int(line)-1] = val['pt']*100

f.close()

In [155]:
graph = collections.defaultdict(list) # from: (to, line)
graph_reverse = collections.defaultdict(list) # to: (from, line)
for i, (from_bus, to_bus) in enumerate(line_to_nodes):
    graph[from_bus].append((to_bus, i))
    graph_reverse[to_bus].append((from_bus, i))

print(graph)

defaultdict(<class 'list'>, {0: [(2409, 0)], 2409: [(19, 1), (2410, 2)], 2410: [(1617, 3), (1616, 4)], 1616: [(2411, 5)], 2358: [(2359, 6), (6748, 8351)], 2030: [(2425, 7), (2430, 34), (2426, 6571)], 6723: [(5592, 8)], 5592: [(2412, 9)], 2425: [(6723, 10), (2361, 28)], 2413: [(8020, 11)], 2414: [(2445, 12), (5619, 10149)], 2033: [(2416, 13)], 2435: [(2415, 14)], 2416: [(2435, 15), (2417, 16)], 1866: [(2033, 17), (2033, 37), (2432, 38)], 1865: [(2439, 18), (2454, 10078)], 2439: [(2418, 19), (2419, 45)], 2419: [(1865, 20), (2440, 46)], 2420: [(5704, 21), (5580, 6759)], 5704: [(2418, 22)], 2421: [(2422, 23)], 2422: [(2423, 24)], 2423: [(2424, 25)], 2424: [(2031, 26), (2413, 27)], 2426: [(6725, 29), (5611, 6573)], 6725: [(2427, 30)], 2427: [(2428, 31)], 2428: [(2429, 32)], 2429: [(2032, 33)], 2430: [(2431, 35)], 2431: [(2032, 36)], 2433: [(2434, 39), (2436, 43)], 2432: [(2437, 40)], 2437: [(2435, 41)], 2436: [(2437, 42)], 2434: [(2438, 44)], 2034: [(2420, 47), (2457, 6718)], 2440: [(2441, 

In [156]:
#DCOPF
# minimize cost_vec.T @ power_generation i.e. equation 5(a)
line_flow_limit = np.full((num_lines, 1), 2589.0) # largest line flow rate
neg_line_flow_limit = np.full((num_lines, 1), -2609.0) # smallest line flow
power_generation_limit = np.full((num_gen, 1), 1164.0) # upper limit of power generation for each generator
neg_power_generation_limit = np.full((num_gen, 1), -0.0) # lower limit

In [157]:
power_generation = cp.Variable(shape=(num_gen, 1)) # shape [num_gen, 1]
line_flow = cp.Variable(shape=(num_lines, 1)) # shape [num_line, 1]
# condition 5(b)
b = np.zeros((num_buses, num_gen), dtype=float) # a matrix to convert power_generation from [x1, x2, x3, x4, x5, x6] to [x1, x2, 0,0,0,0,0,0,0,0,0,x3, ...] of size 30*1
# b @ power_generation = new_power_generation
for i, v in enumerate(generators):
    b[v][i] = 1.0
a = np.zeros((num_buses, num_lines), dtype=float) # this matrix uses ie. 1.0/-1.0 to represent inflow/outflow. 
# for example, line 0 is [1,2], which means power flows from bus 1 to bus 2 at line flow rate x0 at line 0. 
# We would -x0 at index 0 of new line flow and +x0 at index 1 of new line flow. 
# So the new line flow of shape [num_bus, 1] means at each bus/node, what is the power flow (inflow-outflow at the node).
for i, v in enumerate(line_to_nodes):
        a[v[0]][i] = 1.0
        a[v[1]][i] = -1.0

In [158]:
prob = cp.Problem(cp.Minimize(cp.sum(cost_vec @ power_generation)), # eq 5(a)
                [b @ power_generation - load == a @ line_flow, # eqation 5(b) meaning: at each node/bus the power generated at this bus(some bus are generator) + the power inflow(can be negative) == the power demand at the bus
                line_flow <= line_flow_limit, # eq 5(c)
                line_flow >= neg_line_flow_limit, # eq 5(c)
                power_generation <= power_generation_limit, # eq 5(d)
                power_generation >= neg_power_generation_limit]) # eq 5(d)
#prob.solve(solver=cp.CVXOPT)

In [159]:
prob.solve()
print(prob.status)

C:\Users\syq41\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


optimal


In [102]:
sum(power_generation.value)

array([44008.91585859])

In [160]:
problem_nodes = set()
for i, f in enumerate(line_flow.value):
    if f < 0.0:
        from_node, to_node = line_to_nodes[i]
        graph[from_node].remove((to_node, i))
        graph[to_node].append((from_node, i))
        graph_reverse[to_node].remove((from_node, i))
        graph_reverse[from_node].append((to_node, i))
        line_flow.value[i] = -f
line_to_gen = collections.defaultdict(set)
node_to_gen = collections.defaultdict(set)
def dfs(g, n, visited):
    if n in visited: return
    visited.add(n)
    node_to_gen[n].add(g)
    for nei, line in graph[n]:
        line_to_gen[line].add(g)
        dfs(g, nei, visited)

for i, g in enumerate(generators):
    visited = set()
    if power_generation.value[i] >= 0.0001:
        dfs(g, g, visited)
line_prop_mat=np.zeros((num_gen, num_lines), dtype=float)
bus_prop_mat=np.zeros((num_gen, num_buses), dtype=float)
# initially, if node_to_gen only has 1 generator, bus_prop_mat start with 1
for k, v in node_to_gen.items():
    if len(v) == 1:
        idx = list(generators).index(list(v)[0])
        bus_prop_mat[idx][k] = 1.0
for k, v in line_to_gen.items():
    if len(v) == 1:
        idx = list(generators).index(list(v)[0])
        line_prop_mat[idx][k] = 1.0
# visit nodes in topological order
# step 1: calculate bus_prop based on in flowing lines_prop
# step 2: calculate line_prop of out flowing lines
in_degree = collections.defaultdict(int)
for i, v in graph_reverse.items():
    in_degree[i] = len(v)
q = [] # list of nodes with no inflow
topo_order = []
for g in generators:
    if in_degree[g] == 0:
        q.append(g)
while q:
    cur = q.pop(0)
    if len(node_to_gen[cur]) > 1:
        out_total = load[cur][0]
        for nei, out_line in graph[cur]:
            out_total += line_flow.value[out_line]
        for g in node_to_gen[cur]:
            idx = list(generators).index(g)
            if cur == g:
                bus_prop_mat[idx][cur] = power_generation.value[idx]/out_total
            else:
                for nei, in_line in graph_reverse[cur]:
                    if out_total == 0.0:
                        continue
                    bus_prop_mat[idx][cur] += line_flow.value[in_line]*line_prop_mat[idx][in_line]/out_total
        for g in node_to_gen[cur]:
            idx = list(generators).index(g)
            for nei, out_line in graph[cur]:
                line_prop_mat[idx][out_line] = bus_prop_mat[idx][cur]

    topo_order.append(cur)
    for nei, line in graph[cur]:
        in_degree[nei] -= 1
        if in_degree[nei] == 0:
            q.append(nei)
avg_carbon_emission_rate_node = carbon_emission_rate_vec @ bus_prop_mat


C:\Users\syq41\AppData\Local\Temp\ipykernel_13676\1348049050.py:60: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bus_prop_mat[idx][cur] += line_flow.value[in_line]*line_prop_mat[idx][in_line]/out_total
C:\Users\syq41\AppData\Local\Temp\ipykernel_13676\1348049050.py:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bus_prop_mat[idx][cur] = power_generation.value[idx]/out_total


In [105]:
sum(sum(bus_prop_mat))

1978.3480187690864

In [108]:
len(avg_carbon_emission_rate_node)

8870

In [110]:
df_ = pd.read_csv("GIS/CATS_buses.csv")

In [113]:
df_.insert(2, 'avg_carbon_emission', avg_carbon_emission_rate_node)

In [115]:
df_.to_csv('GIS/CATS_avg_bus_carbon_output.csv', index=False)

In [120]:
df = pd.read_csv("GIS/CATS_gens.csv")
df = df[df['Pmax'] != 0.0]
df.insert(2, 'power_gen', power_generation.value)

In [121]:
df.to_csv('GIS/CATS_power_gen_output.csv', index=False)

In [172]:
line_f = line_flow.value.T[0]

In [173]:
d = {'from_bus': branch_from_bus, 'to_bus': branch_to_bus, 'line_flow': line_f}
df = pd.DataFrame(d)

In [182]:
f = open("GIS/CATS_lines_output.json")
sol = json.load(f)
for val in sol['features'][0]['properties']:
    print(val)
f.close()

br_r
rate_a
Comments
br_b
Lon2
Lon1
br_x
kV
Lat1
f_bus
t_bus
CATS_ID
Lat2
transformer
